# Python的闭包

# 1. Python的闭包

① 普通函数的返回值为整数、字符串、列表，但是闭包的外层函数(outer_func)的返回值是内层函数(inner_func)的对象，也可以是内部函数，如果是对象，则可以被调用，通过 对象(参数) 调用，如果是内部函数，则调用装饰器则直接调用内部函数。

② 闭包是一个嵌套函数，内嵌函数可以使用外部函数的入口参数，可以引用外层函数变量，这种内部函数调用外部变量的行为，就叫做闭包。

③ 变量生命周期被延长时因为闭包会使用python中一个魔法属性__closure__，它负责把外部变量生命周期延长，内置函数.__closure__ 有返回值表示该外层函数为闭包，无返回值则不为闭包。

④ 跟普通函数的不同，内嵌函数用了外部函数的变量，使外部变量生命周期被延长了，而普通函数的变量，在进入其他函数时，变量无法再用了。

In [1]:
# 使用外部函数的的入口参数
# 闭包外层函数的返回值为内层函数

def print_func(name):
    def p_name():
        print( "I am " + name )
    print(p_name.__closure__)   # 调用的是内部函数的__closure__，而不是外部函数的__closure__
    return p_name()             

if __name__  ==  '__main__':
    print_func('Tony')    # 直接调用内部函数

(<cell at 0x00000253F92343D8: str object at 0x00000253F9232870>,)
I am Tony


In [2]:
# 使用外部函数的的入口参数
# 闭包外层函数的返回值为内存函数对象

def print_func(name):
    def p_name():
        print( "I am " + name )
    print(p_name.__closure__)
    return p_name  # 返回内层函数对象

if __name__  ==  '__main__':
    print_func('Tony')             # 并不会调用p_name():
    print(print_func('Tony'))      # 打印返回的对象
    
    t = print_func('宝贝入怀')
    t()                            # 此时才会调用内层函数
    
# python处处是对象，函数也可以作为对象，比如t和p_name是指向同一个函数的引用。
# 如果不理解引用，那你就把t跟p_name当成同一个东西，没加括号为函数对象本身，加上括号则代表在此调用方法。
# 调用t即t()则相当于p_name()

(<cell at 0x00000253F9234228: str object at 0x00000253F92412F0>,)
(<cell at 0x00000253F9234228: str object at 0x00000253F92412F0>,)
<function print_func.<locals>.p_name at 0x00000253F9213DC8>
(<cell at 0x00000253F9234228: str object at 0x00000253F9236270>,)
I am 宝贝入怀


In [3]:
# 闭包引用外层函数变量

def outer_func():
    loc_list = []
    def inner_func(name):
        loc_list.append(len(loc_list)+1)
        print("%s loc_list = %s" %(name,loc_list))
    print(inner_func.__closure__)
    return inner_func

clo_func_0 = outer_func()   # 将内层函数对象 重命名为 clo_func_0
print(clo_func_0)
clo_func_0('a')  # 调用一次
clo_func_0('b')  # 调用二次
clo_func_0('c')  # 调用三次  
clo_func_1 = outer_func()
clo_func_0('n')  # 调用一次
clo_func_0('h')  # 调用二次  这些调用都在外层变量 los_list 上运行属性
clo_func_0('j')  # 调用三次

(<cell at 0x00000253F9234CD8: list object at 0x00000253F9185408>,)
<function outer_func.<locals>.inner_func at 0x00000253F9213EE8>
a loc_list = [1]
b loc_list = [1, 2]
c loc_list = [1, 2, 3]
(<cell at 0x00000253F924A348: list object at 0x00000253F9185448>,)
n loc_list = [1, 2, 3, 4]
h loc_list = [1, 2, 3, 4, 5]
j loc_list = [1, 2, 3, 4, 5, 6]


In [4]:
def outer_func1():
    loc_list = []
    def inner_func(name):
        list1 = loc_list   # 每回函数调用，每回都继承原有外部变量 loc_list，这是一个闭包
        list1.append('1')
        print(list1)
    print(inner_func.__closure__)  # 有返回值为闭包
    return inner_func

def outer_func2():
    loc_list = []
    def inner_func(name):
        list1 = []      # 每回函数调用，直接调用这个普通函数，所以每次list1都被初始化为空，而不是继承上一次的list1
        list1.append('1')
        print(list1)
    print(inner_func.__closure__)  # 无返回值为闭包
    return inner_func

f1 = outer_func1() # f1就是内部函数了
f1('dd')  # 每当调用一次，就会往里添加一次1
f1('dd')  # 每当调用一次，就会往里添加一次1
f1('dd')  # 每当调用一次，就会往里添加一次1


f2 = outer_func2()
f2('dd')  # 每当调用一次，就会往里添加一次1
f2('dd')  # 每当调用一次，就会往里添加一次1
f2('dd')  # 每当调用一次，就会往里添加一次1

(<cell at 0x00000253F924A378: list object at 0x00000253F92327C8>,)
['1']
['1', '1']
['1', '1', '1']
None
['1']
['1']
['1']


In [5]:
# 闭包：函数里面定义了一个函数

import time

def f():
    start_time = time.time()
    print("hello")
    time.sleep(1)
    print("world")
    return 100
    
def f1():
    start_time = time.time()
    print("hello")
    print("hello")
    time.sleep(3)
    print("world")
    return 1000

    
def cal_time(func):
    def wrapper():
        start_time = time.time()
        a = func()                   # 用了外部的变量、参数
        print("程序总耗时，{}".format(time.time()-start_time))
        print(wrapper.__closure__)   # 是闭包
    return wrapper
        
func1 = cal_time(f)   # 对第一个函数统计时间
func1()

func2 = cal_time(f1)  # 对第二个函数统计时间
func2()

hello
world
程序总耗时，1.0062108039855957
(<cell at 0x00000253F924A1F8: function object at 0x00000253F92421F8>, <cell at 0x00000253F924A378: function object at 0x00000253F9084CA8>)
hello
hello
world
程序总耗时，3.00880765914917
(<cell at 0x00000253F91FFFA8: function object at 0x00000253F9213AF8>, <cell at 0x00000253F91FFA38: function object at 0x00000253F9084EE8>)


In [6]:
# 函数运行完有返回值，没有返回值就打印空

import time
 
def f():
    start_time = time.time()
    print("hello")
    time.sleep(1)
    print("world")
#    return 100
    
def f1():
    start_time = time.time()
    print("hello")
    print("hello")
    time.sleep(3)
    print("world")
    return 1000

    
def cal_time(func):
    def wrapper():
        start_time = time.time()
        a = func()               # 用了外部的变量、参数
        print("程序总耗时，{}".format(time.time()-start_time))
        return a
    print(wrapper.__closure__)   # 是闭包
    return wrapper
        
func1 = cal_time(f)              # 对第一个函数统计时间
func1()
print(func1())

func2 = cal_time(f1)             # 对第二个函数统计时间
func2()
print(func2())

(<cell at 0x00000253F924A0D8: function object at 0x00000253F92423A8>,)
hello
world
程序总耗时，1.008488416671753
hello
world
程序总耗时，1.0043680667877197
None
(<cell at 0x00000253F9234F78: function object at 0x00000253F924C0D8>,)
hello
hello
world
程序总耗时，3.00726056098938
hello
hello
world
程序总耗时，3.006143808364868
1000


In [7]:
# 函数有传入参数的情况

import time

def f(a):
    start_time = time.time()
    print("hello")
    time.sleep(1)
    print("world")
    return a+9
    
def f1(a,b,c):
    start_time = time.time()
    print("hello")
    print("hello")
    time.sleep(3)
    print("world")
    return a+b+c+1000

    
def cal_time(func):
    def wrapper(a):
        start_time = time.time()
        result = func(a)         # 用了外部的变量、参数
        print("程序总耗时，{}".format(time.time()-start_time))
        return result
    print(wrapper.__closure__)   # 是闭包
    return wrapper
        
func1 = cal_time(f)              # 对第一个函数统计时间
func1(10)
print(func1(10))

(<cell at 0x00000253F924AF48: function object at 0x00000253F924C5E8>,)
hello
world
程序总耗时，1.0029902458190918
hello
world
程序总耗时，1.0046665668487549
19


In [8]:
# 函数传入不定长参数

import time
 
# 如果有一百个这样的函数要统计时间，就比较麻烦
def f(a):
    start_time = time.time()
    print("hello")
    time.sleep(1)
    print("world")
    return a+9
    
def f1(a,b,c):
    start_time = time.time()
    print("hello")
    print("hello")
    time.sleep(3)
    print("world")
    return a+b+c+1000

    
def cal_time(func):
    def wrapper(*a):
        start_time = time.time()
        result = func(*a) # 用了外部的变量、参数
        print("程序总耗时，{}".format(time.time()-start_time))
        return result
    print(wrapper.__closure__)   # 是闭包
    return wrapper
        
func1 = cal_time(f)   # 对第一个函数统计时间
func1(10)
print(func1(10))

func2 = cal_time(f1)  # 对第二个函数统计时间
func2(10,100,1000)
print(func2(10,100,1000))

(<cell at 0x00000253F924A8B8: function object at 0x00000253F9242EE8>,)
hello
world
程序总耗时，1.0104401111602783
hello
world
程序总耗时，1.011256456375122
19
(<cell at 0x00000253F9234D38: function object at 0x00000253F924C048>,)
hello
hello
world
程序总耗时，3.0024991035461426
hello
hello
world
程序总耗时，3.008516311645508
2110


In [9]:
# 闭包陷阱，看似闭包，实际上是闭包

def my_func(*args):
    fs = []
    for i in range(3):
        def func():
            return i * i         # 内部函数使用了外部变量 i，所以是闭包。
        print(func.__closure__)  # 有返回值，是闭包
        fs.append(func)
    return fs
 
fs1,fs2,fs3 = my_func()          # 表示 my_func() 运行一次，由于for循环了三次，所以fs中有三个元素，这三个元素赋值为fs1、fs2、fs3
                                 # my_func() 运行中，不会进入 return i * i ，仅仅运行添加函数地址 fs.append(func)

print(fs1)
print(fs2)
print(fs3)                       # 相当于实例化了三次，三个不同的实例化对象，所以地址不一样

print(fs1())                     # fs1、fs2、fs3 相当于都是内置函数 func 对象，因此fs1()为直接调用内置函数方法
print(fs2())                     # 当运行内置函数方法时，此时的 i 在前面的循环结束后 值为2
print(fs3())                     # 此时调用fs1()相当于调用

(<cell at 0x00000253F924A2E8: int object at 0x00007FFCE768A170>,)
(<cell at 0x00000253F924A2E8: int object at 0x00007FFCE768A190>,)
(<cell at 0x00000253F924A2E8: int object at 0x00007FFCE768A1B0>,)
<function my_func.<locals>.func at 0x00000253F924C678>
<function my_func.<locals>.func at 0x00000253F924C558>
<function my_func.<locals>.func at 0x00000253F924CCA8>
4
4
4


In [10]:
# 闭包陷阱，看似闭包，实际上不是闭包

def my_func(*args):
    fs = []
    for i in range(3):
        def func(_i = i):        # i 作为值传入给_i变量，内部函数实际上没有用外部变量 i
            return _i * _i
        print(func.__closure__)  # 无返回值，不是闭包。
        fs.append(func)
    return fs
 
fs1,fs2,fs3 = my_func()  

print(fs1())     
print(id(fs1))   
print(fs2())
print(id(fs2))
print(fs3())  
print(id(fs3))

None
None
None
0
2559685479448
1
2559685476712
4
2559685439352
